<a href="https://colab.research.google.com/github/ulya1202/NLP/blob/main/Fake_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Generating Shakespearean Text Using a Character RNN

In [ ]:
import tensorflow as tf
shakespeare_url = "https://homl.info/shakespeare"

filepath = tf.keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [ ]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [ ]:
text_vec_layer  = tf.keras.layers.TextVectorization(split='character',
                                                    standardize = 'lower')#. vergul lazim
text_vec_layer.adapt(shakespeare_text)
encoded = text_vec_layer(shakespeare_text)#-elave listun ichinden cixaririq tex vece gonderririk

In [ ]:
encoded -=2 #unkw ve paddingi sil - qiymet ignore edir onunchun 0-2 ve.s
vocab_size = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)


In [ ]:
dataset_size #listin ichinen cixarmasaq 1 olar

1115394

In [ ]:
#input -> Before we proceed any furthe
#output -> efore we proceed any further

In [ ]:
def to_dataset(sequence , lenght ,seed =None,shuffle = False,batch_size = 32):
  ds = tf.data.Dataset.from_tensor_slices(sequence)
  ds = ds.window(lenght+1,shift=1,drop_remainder = True) #qissa daha suretlidi amma kokhne elaqeler unudulur
  ds = ds.flat_map(lambda window_ds: window_ds.batch(lenght+1)) #hamisini birleshdirir
  if shuffle :
    ds = ds.shuffle(100_000,seed=seed) #batchleri qarishdir
  ds = ds.batch(batch_size)
  return ds.map(lambda window:(window[:,:-1],window[:,1:]))

In [ ]:
lenght = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000],lenght=lenght,shuffle=True,
                       seed=42)
valid_set = to_dataset(encoded[1_000_000:1_060_000],lenght=lenght)
test_set = to_dataset(encoded[1_060_000:],lenght=lenght)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = vocab_size,output_dim=16), #bir herifi ifade etmek uchun kifayetdi
    tf.keras.layers.GRU(128,return_sequences = True), #y- i return etsin  hamisinda cixir
    tf.keras.layers.Dense(vocab_size,activation="softmax") #her neyron bir herif
])
model.compile(loss="sparse_categorical_crossentropy",optimizer="nadam",
              metrics = ["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model.keras",
    monitor="val_accuracy",
    save_best_only = True)

history = model.fit(train_set,validation_data=valid_set,epochs=1,
                    callbacks=[model_ckpt])

  31244/Unknown 401s 12ms/step - accuracy: 0.5485 - loss: 1.4933

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 417s 13ms/step - accuracy: 0.5485 - loss: 1.4933 - val_accuracy: 0.5337 - val_loss: 1.6072


In [ ]:
shakespeare_model = tf.keras.Sequential([ #test uchun
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X-2), #paddingle baqli pad unk
    model
])

In [ ]:
y_proba = shakespeare_model.predict(tf.constant(["To be or not to b"]))[0,-1] #en axirinci herifi qaytarmisr !!!
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred+2] #unk ve pad

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step


np.str_('e')

#Generating Fake Shakespeare text

In [ ]:
log_probas = tf.math.log([[0.5,0.3,0.2]])
tf.random.categorical(log_probas,num_samples=8)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 0, 0, 1, 0, 0, 0, 1]])>

In [ ]:
#"Bu log ehtimallardan istifadə et, və 8 dəfə təsadüfi kateqoriya seç"

#Yəni, 0.5 ehtimalla 0, 0.3 ilə 1, 0.2 ilə 2 gələcək.

In [ ]:
shakespeare_model.predict(tf.constant(['hello my dearr']))[0].shape #cumledeki charekterler setirde -- yeni h-dan sonra hansi l dan sonra hansi ve.s --> butun vocabulary uzre

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


(14, 39)

In [ ]:
shakespeare_model.predict(tf.constant(['hello my dearr']))[0,-1:]#noqtesizde eyni neticeni verdi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


array([[3.69505249e-02, 2.75135607e-01, 2.76962855e-05, 1.36351194e-02,
        3.04775499e-02, 1.35534242e-01, 2.32875591e-05, 5.54100983e-02,
        2.10383977e-03, 4.07455673e-06, 4.97320257e-02, 2.53603503e-04,
        1.09040855e-04, 2.61573605e-02, 1.83994329e-04, 5.41657768e-02,
        1.11590640e-03, 2.45396625e-02, 2.06100231e-04, 4.77084331e-03,
        1.47981060e-04, 3.30972427e-04, 1.33541116e-05, 2.35887337e-03,
        8.31076704e-07, 1.48155232e-05, 2.57752016e-02, 2.22173974e-01,
        1.06029818e-02, 3.89179611e-03, 2.26366110e-02, 1.50033308e-03,
        7.92848459e-06, 9.41146993e-07, 6.91933155e-06, 1.17855791e-07,
        1.12989778e-08, 5.74992365e-09, 1.53976040e-12]], dtype=float32)

In [ ]:
def next_char(text,temperature):
  text = tf.constant([text])
  y_proba = shakespeare_model.predict(text)[0,-1:]# niye iki noqte
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)[0,0]# iki moteroized qurtaramq uchun
  return text_vec_layer.get_vocabulary()[char_id+2] #chartcertid tapidi

In [ ]:
def extent_text(text,chars=50,temperature = 1):
  for _ in range(chars):
    text +=next_char(text,temperature)
  return text

In [ ]:
extent_text('to be or not to be',chars = 100,temperature = 1)